In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [5]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [6]:
EPOCHS     = 40
BATCH_SIZE = 128

## Load data

In [7]:
#img to tensor
transpose = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',train=True,download=True,transform=transpose),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',train=False,transform=transpose),
    batch_size=BATCH_SIZE, shuffle=True)

## 뉴럴넷

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # [128, 10, 24, 24] = [배치사이즈, 아웃풋필터10, (28-5)/stride(1) + 1] -> # [128, 10, 12, 12]  
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) #[128, 20, 8, 8] -> #[128, 20, 4, 4]
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [11]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [12]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302907
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.230491
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.650841
[1] Test Loss: 0.3028, Accuracy: 91.71%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.485995
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.362546
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.555973
[2] Test Loss: 0.1787, Accuracy: 94.80%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.512738
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.403041
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.340825
[3] Test Loss: 0.1396, Accuracy: 95.76%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.382072
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.343012
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.324848
[4] Test Loss: 0.1161, Accuracy: 96.34%
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.416307
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.314659
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.378175
[5] Test Loss: 0.1029, Accuracy: 96.68%
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.220242
Train Epoch: 6 [25600/60000 (4